In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Download and extract the dataset
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_file = tf.keras.utils.get_file("cats_and_dogs_filtered.zip", url, extract=True)
base_dir = os.path.join(os.path.dirname(zip_file), 'cats_and_dogs_filtered')

# Set up directories for training and validation data
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Set up data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# Load pre-trained models
vgg16_base = VGG16(weights='imagenet', include_top=False)
resnet50_base = ResNet50(weights='imagenet', include_top=False)
resnet152_base = ResNet152(weights='imagenet', include_top=False)

# Add custom classifier layers on top of the pre-trained models
def add_custom_layers(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)  # Output layer with binary classification
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Create models with custom layers
vgg16_model = add_custom_layers(vgg16_base)
resnet50_model = add_custom_layers(resnet50_base)
resnet152_model = add_custom_layers(resnet152_base)

# Compile models
# sgd = SGD(lr=0.001, momentum=0.9)
from tensorflow.keras.optimizers import SGD


# sgd = legacy.SGD(lr=0.001, momentum=0.9)
sgd = SGD(learning_rate=0.001, momentum=0.9)


vgg16_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
resnet50_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
resnet152_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

trainable_variables = []
for model in [vgg16_model, resnet50_model, resnet152_model]:
    trainable_variables.extend(model.trainable_variables)

sgd.build(trainable_variables)


# Train the models
num_epochs = 5

vgg16_model.fit(train_generator, epochs=num_epochs, validation_data=valid_generator)
resnet50_model.fit(train_generator, epochs=num_epochs, validation_data=valid_generator)
resnet152_model.fit(train_generator, epochs=num_epochs, validation_data=valid_generator)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5
63/63 [==============================] - 14s 213ms/step - loss: 0.6058 - accuracy: 0.6480 - val_loss: 0.4259 - val_accuracy: 0.8010
Epoch 2/5
63/63 [==============================] - 13s 206ms/step - loss: 0.2603 - accuracy: 0.9035 - val_loss: 0.1209 - val_accuracy: 0.9570
Epoch 3/5
63/63 [==============================] - 13s 207ms/step - loss: 0.1450 - accuracy: 0.9465 - val_loss: 0.1439 - val_accuracy: 0.9480
Epoch 4/5
63/63 [==============================] - 13s 207ms/step - loss: 0.1015 - accuracy: 0.9595 - val_loss: 0.1029 - val_accuracy: 0.9560
Epoch 5/5
63/63 [==============================] - 13s 208ms/step - loss: 0.0442 - accuracy: 0.9860 - val_loss: 0.0961 - val_accuracy: 0.9610
Epoch 1/5
63/63 [==============================] - 16s 148ms/step - loss: 0.2457 - accuracy: 0.9145 - val_loss: 0.7354 - val_accuracy: 0.5000
Epoch 2/5
63/63 [==============================] - 6s 99ms/step 